In [925]:
import pandas as pd
import numpy as np
import re
import pickle

from pymongo import MongoClient

import sys
sys.path.insert(0, '../src/features/')
sys.path.insert(0, '../src/data/')
%load_ext autoreload

%autoreload 1
%aimport spacy_lemmer
import spacy_lemmer

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [926]:
from clean_book import book_cleaner


In [939]:
client=MongoClient()
db = client['proj_4_gutenberg']
collection = db['gutenberg_books']

book_ids = collection.find({}, {'_id':1, 'title':1, 'author':1})
books_for_dict = {}
book_nums = []
book_titles = []
book_authors = []
for book in book_ids:
    #Imp of Earn (15399) inputs many proper nouns, Moby Dick (2148) and War and Peace(2600) and Jane Eyre (1260) are too long, 
    #and is breaking the lemmatizer, so for now, I am removing them.
#     if book['_id'] != '2148' and book['_id']!='15399' and book['_id']!='2600' and book['_id']!='1260':
#         book_nums.append((book['_id']))
    book_nums.append((book['_id']))
    book_titles.append((book['title']))
    book_authors.append((book['author']))
    
    url = 'https://www.gutenberg.org/ebooks/%s' % book['_id']
    books_for_dict[book['title']]=[book['author'],url]

    
print(books_for_dict)    

{'Pride and Prejudice': ['Jane Austen', 'https://www.gutenberg.org/ebooks/1342'], 'Frankenstein': ['Mary Wollstonecraft (Godwin) Shelley', 'https://www.gutenberg.org/ebooks/84'], 'Heart of Darkness': ['Joseph Conrad', 'https://www.gutenberg.org/ebooks/219'], 'A Modest Proposal': ['Jonathan Swift', 'https://www.gutenberg.org/ebooks/1080'], 'A Tale of Two Cities': ['Charles Dickens', 'https://www.gutenberg.org/ebooks/98'], 'The Importance of Being Earnest': ['Oscar Wilde', 'https://www.gutenberg.org/ebooks/15399'], "Alice's Adventures in Wonderland": ['Lewis Carroll', 'https://www.gutenberg.org/ebooks/76'], 'Moby Dick; or The Whale': ['Herman Melville', 'https://www.gutenberg.org/ebooks/2148'], 'Voyage to Jupiter': ['David Morrison', 'https://www.gutenberg.org/ebooks/58915'], 'The Adventures of Sherlock Holmes': ['Arthur Conan Doyle', 'https://www.gutenberg.org/ebooks/160'], 'The Souls of Black Folk': ['W. E. B. Du Bois', 'https://www.gutenberg.org/ebooks/2097'], 'Metamorphosis': ['Franz

In [942]:
with open('../src/visualization/flask_app/dict_with_urls.pk', "wb") as output_file:
    pickle.dump(books_for_dict, output_file)

In [482]:
books_for_lemming = [book_cleaner(str(book)) for book in book_nums]
spacy_lemmed_text = [spacy_lemmer.spacy_lem_text(book) for book in books_for_lemming]

Pride and Prejudice 499000
Frankenstein 415792
Heart of Darkness 208333
A Modest Proposal 19173
A Tale of Two Cities 499000
The Importance of Being Earnest 116522
Alice's Adventures in Wonderland 133809
Moby Dick; or The Whale 499000
Voyage to Jupiter 389891
The Adventures of Sherlock Holmes 499000
The Souls of Black Folk 395540
Metamorphosis 117359
The Yellow Wallpaper 30931
The Earth Quarter 109426
War and Peace 499000
Grimms' Fairy Tales 499000
Dracula 499000
Jane Eyre 499000
The Picture of Dorian Gray 424422
Walden, and On The Duty Of Civil Disobedience 499000
The Count of Monte Cristo 499000
Leaves of Grass 8758
Autographs for Freedom 295429
Great Expectations 499000
Peter Pan 250581
Dubliners 364177
The Narrative of the Life of Frederick Douglass 217104
Pygmalion 184851
The Romance of Lust 499000
Ulysses 499000
Siddhartha 213768
A Christmas Carol 155526
Treasure Island 355349
Leviathan 499000
Legendary Tales of the Highlands (Volume 1 of 3) 312155
Wuthering Heights 499000
The Ess

In [489]:
len(spacy_lemmed_text)

76

In [494]:
with open('../data/processed/spacy_lemmed_train_text_500000.pickle', "wb") as output_file:
    pickle.dump(spacy_lemmed_text, output_file)

## Add new book from Mongo

In [254]:
# with open('../data/processed/spacy_lemmed_train_text_500000.pickle', "rb") as input_file:
#     lemmed_text = pickle.load(input_file)

In [255]:
# book_for_lemming = book_cleaner(964)
# spacy_lemmed_text = spacy_lemmer.spacy_lem_text(book_for_lemming)
# lemmed_text.append(spacy_lemmed_text)

The Merry Adventures of Robin Hood 499000
429113


In [260]:
# with open('../data/processed/spacy_lemmed_train_text_500000.pickle', "wb") as output_file:
# #     pickle.dump(lemmed_text, output_file)

### end add new book

In [261]:
# with open('../data/processed/lemmed_train_data.pickle', "wb") as output_file:
#     pickle.dump(lemmed_text, output_file)

In [495]:
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import scale
from sklearn.cluster import KMeans

from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF
from sklearn.metrics.pairwise import cosine_similarity

In [496]:
with open('../data/processed/spacy_lemmed_train_text_500000.pickle', "rb") as input_file:
    lemmed_text = pickle.load(input_file)


In [497]:
len(lemmed_text)

76

In [498]:
lemmed_text[-1]

' s a great , tall , merry fellow that roam the greenwood and join in homely sport , and sit beside the at merry feast , which same beareth the name of the proudest of the plantagenets — of the \'s heart . beside these be a whole host of knight , priest , noble , burgher , yeoman , page , lady , lass , landlord , beggar , peddler , and what not , all live the merry of merry life , and all bind by nothing but a few odd strand of certain old ballad ( snip and clip and tie together again in a score of knot ) which draw these jocund fellow here and there , sing as go . here will find a hundred dull , sober , jog place , all trick out with flower and what not , till no one would know in fanciful dress . and here be a country bear a well - know name , wherein no chill mist press upon spirit , and no rain fall but what roll off back like shower off the back of sleek drake ; where flower bloom forever and bird be always sing ; where every fellow hath a merry catch as travel the road , and ale 

In [499]:
tfidf= TfidfVectorizer(min_df = .1, max_df = .9, stop_words='english', ngram_range=(1,2))
doc_word = tfidf.fit_transform(lemmed_text)

In [658]:
with open('../src/models/tf_idf.pk', 'wb') as write_file:
    pickle.dump(tfidf, write_file)

## Truncated SVD

In [860]:
lsa = TruncatedSVD(50)
doc_topic = lsa.fit_transform(doc_word)
lsa.explained_variance_ratio_
len(doc_topic)

76

In [861]:
sum(lsa.explained_variance_ratio_)


0.8357929155826891

### lsa topics for flask

In [657]:
with open('../src/models/doc_lsa.pk','wb') as output_file:
    pickle.dump(doc_topic, output_file)

### end lsa topics for flask

In [504]:
!ls ../src/models/

__init__.py      lsa.pk           tf_idf.pk
doc_lsa.pk       predict_model.py train_model.py


### lsa for flask app

In [659]:
with open('../src/models/lsa.pk', 'wb') as output_file:
    pickle.dump(lsa, output_file)

### end lsa for flask app

In [21]:
with open('../src/models/tfidf_73.pickle', "wb") as output_file:
    pickle.dump(tfidf, output_file)

In [22]:
# with open('../src/models/lsa_73.pickle', "wb") as output_file:
#     pickle.dump(lsa, output_file)

In [505]:
def display_topics(model, feature_names, no_top_words, topic_names=None):
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

display_topics(lsa, tfidf.get_feature_names(), 10)


Topic  0
father, dear, oh, mother, yes, boy, girl, sir, lady, sister

Topic  1
thou, power, hath, thee, opinion, thy, nature, subject, government, law

Topic  2
sister, dear, oh, lady, girl, pleasure, manner, feeling, father, aunt

Topic  3
thou, thee, thy, mother, dear, sister, doth, unto, hath, oh

Topic  4
ship, captain, sea, boat, island, shore, reply, sail, voyage, king

Topic  5
slave, slavery, boy, mother, master, girl, dollar, tenement, negro, nigger

Topic  6
sir, gentleman, tenement, car, train, street, thou, job, sister, dollar

Topic  7
hath, sir, boy, girl, common, sect, government, power, gentleman, case

Topic  8
father, king, wife, town, princess, forest, mother, food, prince, kingdom

Topic  9
tenement, dollar, car, sister, job, planet, satellite, machine, cent, till

Topic  10
slave, power, slavery, tone, government, sister, sect, law, soldier, bridge

Topic  11
bridge, satellite, soldier, planet, oh, reply, girl, thou, tone, slave

Topic  12
boy, rope, island, cliff

### book title for flask

In [271]:
# with open('../src/visualization/flask_app/book_titles.pk', 'wb') as write_file:
#     pickle.dump(book_titles, write_file)

### end book title for flask

In [856]:
lsa.singular_values_

array([4.48799348, 1.78269623, 1.69741493, 1.50249414, 1.27874215,
       1.20368918, 1.15430188, 1.14355654, 1.12604494, 1.08733014,
       1.07252015, 1.02781986, 1.01299016, 1.00542779, 0.99044695,
       0.98287071, 0.97424237, 0.96618458, 0.95966729, 0.95351721,
       0.94009545, 0.93085345, 0.92805435, 0.91937735, 0.91557247,
       0.90335867, 0.8951085 , 0.87710213, 0.86129225, 0.85041063,
       0.84240166, 0.83298431, 0.82094556, 0.81516365, 0.8082149 ,
       0.80710815, 0.79048573, 0.78897681, 0.78084691, 0.76888717,
       0.7636417 , 0.75194695, 0.74755189, 0.74344491, 0.73682769,
       0.72989699, 0.71936921, 0.71599117, 0.70591294, 0.69946329])

In [836]:
len(doc_topic)

76

I would guess that the reason Jane Eyre, Sherlock Holmes, Dubliners come up so often is that the first column is so strong. It may be really messing up the data.

In [898]:
Vt = pd.DataFrame(doc_topic.round(5),
             index = book_titles,
             columns = lsa.singular_values_)
# Vt.sort_values('component_5', ascending=False))
list(Vt)
Vt.sort_values(4.487993476941114, ascending=False)

,4.487993476941114,1.7826962336953194,1.6974149303832664,1.5024941375452643,1.278742215411877,1.203689028750395,1.1543020368248025,1.1435539098635497,1.1260456272370334,1.0873309006341205,...,0.7612217106862065,0.7512060374327979,0.7482305808670181,0.743029361088888,0.7377168611078793,0.7269994734745117,0.7226662110098813,0.7157661810824898,0.7053396009614971,0.6983438163709833
Jane Eyre,0.74337,-0.10906,0.07271,0.01418,-0.08356,-0.05794,0.01183,0.03478,-0.10429,-0.05994,...,-0.12362,-0.01359,-0.11926,0.11801,-0.09603,-0.00595,-0.01312,-0.05869,-0.06876,-0.02555
Wuthering Heights,0.73526,-0.04441,0.07135,0.08263,0.03264,0.01701,0.00211,-0.06689,-0.09224,-0.05237,...,-0.10439,-0.13142,-0.09026,0.09200,0.02665,-0.00418,-0.05209,-0.11733,-0.11926,0.01205
Hard Times,0.72851,-0.04022,0.11562,0.12628,-0.03260,-0.00964,0.17709,0.11071,-0.00657,-0.10292,...,-0.16601,-0.01967,0.00340,-0.05050,-0.12347,0.04136,0.15999,-0.02311,-0.06045,-0.01288
The Adventures of Sherlock Holmes,0.70934,-0.15757,0.03006,-0.08483,-0.09203,-0.18507,0.17143,0.08720,-0.01852,-0.10135,...,0.07773,-0.01272,0.06636,-0.02252,0.01094,0.06954,-0.05084,-0.00042,-0.02435,0.01791
Northanger Abbey,0.70260,0.11423,0.42691,0.12075,0.06927,-0.05632,0.02354,-0.13909,-0.07967,0.09622,...,0.02101,-0.02826,-0.02315,-0.03234,-0.01797,0.03623,-0.01517,-0.00945,-0.03735,-0.01575
Great Expectations,0.69423,-0.16453,0.09773,0.02324,0.02167,0.00555,0.12904,0.10704,-0.01814,-0.04905,...,-0.08013,0.06628,-0.01344,-0.00857,0.02507,-0.04903,-0.02808,-0.02493,0.21235,0.00560
A Tale of Two Cities,0.68544,-0.09911,-0.01344,-0.02380,0.04058,-0.08373,0.14764,0.04767,0.05893,-0.16568,...,-0.06473,0.04078,-0.07723,-0.18803,-0.12908,0.15076,0.09434,0.02714,-0.04163,-0.03449
Little Women,0.68097,-0.27484,0.14373,0.16998,-0.09420,0.17324,-0.10806,0.06541,0.00660,0.11370,...,-0.03774,0.03532,0.00236,0.06814,0.00836,-0.01839,-0.00099,-0.00473,0.12431,-0.00966
Emma,0.67983,0.10974,0.47547,0.13434,0.05276,-0.02983,-0.00321,-0.07805,-0.07764,0.11557,...,0.08077,-0.04561,0.00430,-0.07160,-0.05286,0.05372,0.02346,0.01185,-0.04033,-0.00526
Mansfield Park,0.67185,0.14262,0.47974,0.11915,0.06578,-0.03601,0.02260,-0.13562,-0.09770,0.15149,...,0.07565,-0.00838,-0.00629,-0.00677,-0.00340,0.00632,-0.02699,0.02867,-0.01132,0.01319


## Test Book 

In [414]:
# with open('../src/models/tfidf_73.pickle', "rb") as input_file:
#     tfidf = pickle.load(input_file)
    
# with open('../src/models/lsa_73.pickle', "rb") as input_file:
#     lsa = pickle.load(input_file)

In [612]:
!ls ../data/external/

anne_frank.txt      enchanted_burro.txt ninthes.txt         whales.txt
becoming.txt        invisible_man.txt   pkd_exc.txt
don_quix.txt        never_let_me_go.txt three_body.txt


In [827]:
file_loc = '/Users/mbarry/Documents/Coding/ds/metis/projects/gutenberg_recommender/data/external/never_let_me_go.txt'
title_of_test = 'anne_frank'

In [828]:
import book_to_compare
lemmed_test = book_to_compare.new_test_book(file_loc, title_of_test)
print(lemmed_test)

15362
[' name be be thirty - one year old , and have be a carer now for over eleven year . that sound long enough , know , but actually want to go on for another eight month , until the end of this year . that will make almost exactly twelve year . now know be a carer so long be not necessarily because think be fantastic at what do . there be some really good carer who have be tell to stop after just two or three year . and can think of one carer at least who go on for all of fourteen year despite be a complete waste of space . so be not try to boast . but then do know for a fact have be please with work , and by and large , have too . donor have always tend to do much good than expect . recovery time have be impressive , and hardly any of have be classify as agitate , even before fourth donation . okay , maybe be boast now . but mean a lot to , be able to do work well , especially that bit about donor stay calm . have develop a kind of instinct around donor . know when to hang around 

In [829]:
test_tfidf = tfidf.transform(lemmed_test)
test_lsa = lsa.transform(test_tfidf)
print(test_lsa)


[[ 2.29920078e-01 -1.08978319e-01 -3.18297513e-02 -2.95072798e-02
  -5.30976071e-02 -8.12349801e-03 -1.86598626e-02  2.73550521e-02
  -1.53593528e-03  8.27361642e-02 -1.16292609e-03 -2.07594939e-02
   2.39984951e-02  3.04318767e-02  5.43325662e-03 -3.19360970e-02
  -3.76214855e-02  1.16418919e-02 -4.03145733e-02  4.43521254e-03
  -1.58728945e-02  5.06157013e-03  1.50213759e-02  9.64883596e-03
   1.37835447e-02 -1.06314677e-02 -5.19234677e-03  2.24812887e-02
  -1.47984373e-02 -2.88285654e-03  6.83467725e-02 -6.64375658e-03
  -1.52239337e-05 -1.30572811e-02  2.06960840e-02  4.25665453e-02
  -1.23135589e-02 -3.09919966e-02  7.46364392e-03 -1.37393044e-02
   5.50954209e-03  1.39127294e-02  8.59140163e-03  3.51911331e-04
   3.48870542e-02 -3.33274143e-02 -1.75770465e-02 -7.53350186e-03
  -1.30332865e-02  2.85142850e-02]]


In [830]:
client=MongoClient()
db = client['proj_4_gutenberg']
collection = db['gutenberg_books']

book_ids = collection.find({}, {'_id':1, 'title':1, 'author':1})
book_nums = []
book_titles = []
book_authors= []
for book in book_ids:
    #Imp of Earn (15399) inputs many proper nouns, Moby Dick (2148) and War and Peace(2600) and Jane Eyre (1260) are too long, 
    #and is breaking the lemmatizer, so for now, I am removing them.
#     if book['_id'] != '2148' and book['_id']!='15399' and book['_id']!='2600' and book['_id']!='1260':
#         book_nums.append((book['_id']))
    book_nums.append((book['_id']))
    book_titles.append((book['title']))
    book_authors.append((book['author']))

In [832]:
book_titles.append(title_of_test)

In [837]:
from sklearn.preprocessing import scale
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D 


doc_for_k = np.concatenate([doc_topic, test_lsa], axis=0)
doc_for_k

array([[ 0.66845112,  0.1730879 ,  0.53359838, ...,  0.01562875,
         0.01335498,  0.01750593],
       [ 0.6199776 ,  0.23604412,  0.03440312, ...,  0.09013783,
         0.15460635, -0.14260102],
       [ 0.51337351, -0.13191966, -0.21807387, ..., -0.06547681,
         0.03880261,  0.02160435],
       ...,
       [ 0.4221728 ,  0.26789014, -0.01929795, ...,  0.020529  ,
        -0.09517843, -0.01230774],
       [ 0.26591084,  0.25615621, -0.4809094 , ..., -0.0047777 ,
         0.0337663 ,  0.0102904 ],
       [ 0.22992008, -0.10897832, -0.03182975, ..., -0.0075335 ,
        -0.01303329,  0.02851428]])

In [838]:
len(doc_for_k)

77

In [839]:
doc_for_k[34]

array([ 5.04254307e-01,  2.65646683e-01, -3.29253018e-01,  5.10319064e-01,
        2.83644165e-02, -6.65074658e-02,  1.19787213e-01, -1.92769617e-04,
       -7.80143069e-02,  8.76880402e-03,  4.21033922e-02,  4.40919015e-02,
       -1.89285092e-03, -6.90656517e-02, -6.92959672e-04, -1.41175715e-02,
       -2.85960584e-02, -5.31042503e-03, -1.86357358e-02,  4.87719919e-02,
       -1.61694620e-02,  5.87861916e-02, -1.79776545e-02,  6.72035345e-02,
       -1.26071124e-02,  2.06577856e-04, -2.35770889e-03,  2.91446003e-02,
       -8.02634547e-02,  1.44081284e-02,  1.97585812e-02,  3.70739382e-02,
       -8.93948092e-03,  9.07249851e-03,  2.19552554e-02, -3.59765133e-03,
        3.73468855e-02,  8.18018362e-02, -2.47577797e-02, -3.04924337e-02,
        6.11205438e-02, -1.00005314e-02, -4.85309671e-02,  5.45367537e-02,
        4.00736929e-02,  1.47224322e-03, -3.18100631e-02,  1.06106440e-02,
        8.52078735e-02, -6.29514450e-02])

In [840]:
test_topics = doc_for_k[-1]
first = 0
first_i = 0
sec = 0
sec_i = 0
third = 0
third_i = 0
for i, topic in enumerate(test_topics):
    if topic > first:
        third = sec
        third_i = sec_i
        sec = first
        sec_i = first_i
        first = topic
        first_i = i
    elif topic > sec:
        third = sec
        third_i = sec_i
        sec = topic
        sec_i = i
    else:
        if topic > third:
            third = topic
            third_i = i
print(first, first_i)
print(sec, sec_i)
print(third, third_i)

0.22992007789133306 0
0.08273616423630453 9
0.06834677245619292 30


In [844]:
a= cosine_similarity(doc_for_k)

In [842]:
top_cos_sim = np.argsort(a[-1])[:-1]
print(book_titles[top_cos_sim[-1]])
print(book_titles[top_cos_sim[-2]])
print(book_titles[top_cos_sim[-3]])

The Adventures of Tom Sawyer, Complete
Great Expectations
Little Women


In [740]:
num_clusters = 15
km = KMeans(n_clusters=num_clusters,random_state=10,n_init=5) # n_init, number of times the K-mean algorithm will run
km.fit(doc_for_k)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=15, n_init=5, n_jobs=None, precompute_distances='auto',
    random_state=10, tol=0.0001, verbose=0)

In [741]:
group_labels = km.labels_
for group,title in zip(group_labels, book_titles):
    if group == 5:
        print(title, group)

Legendary Tales of the Highlands (Volume 1 of 3) 5
Meditations 5
The Scarlet Letter 5
The Merry Adventures of Robin Hood 5
don_quix 5


In [742]:
(book_titles)

['Pride and Prejudice',
 'Frankenstein',
 'Heart of Darkness',
 'A Modest Proposal',
 'A Tale of Two Cities',
 'The Importance of Being Earnest',
 "Alice's Adventures in Wonderland",
 'Moby Dick; or The Whale',
 'Voyage to Jupiter',
 'The Adventures of Sherlock Holmes',
 'The Souls of Black Folk',
 'Metamorphosis',
 'The Yellow Wallpaper',
 'The Earth Quarter',
 'War and Peace',
 "Grimms' Fairy Tales",
 'Dracula',
 'Jane Eyre',
 'The Picture of Dorian Gray',
 'Walden, and On The Duty Of Civil Disobedience',
 'The Count of Monte Cristo',
 'Leaves of Grass',
 'Autographs for Freedom',
 'Great Expectations',
 'Peter Pan',
 'Dubliners',
 'The Narrative of the Life of Frederick Douglass',
 'Pygmalion',
 'The Romance of Lust',
 'Ulysses',
 'Siddhartha',
 'A Christmas Carol',
 'Treasure Island',
 'Leviathan',
 'Legendary Tales of the Highlands (Volume 1 of 3)',
 'Wuthering Heights',
 'The Essays of Montaigne, Complete',
 'Les Misérables',
 'Songs of Innocence and Songs of Experience',
 'A Stu

In [743]:
group_labels = km.labels_
test_group = ''
for group,title in zip(group_labels, book_titles):
    if title == title_of_test:
        test_group = group
        print(title, test_group)
for group,title in zip(group_labels, book_titles):
    if group == test_group:
        print(title, group)

don_quix 5
Legendary Tales of the Highlands (Volume 1 of 3) 5
Meditations 5
The Scarlet Letter 5
The Merry Adventures of Robin Hood 5
don_quix 5


## NMF

In [845]:
from sklearn.decomposition import NMF


In [919]:
nmf_model = NMF(7)
doc_topic = nmf_model.fit_transform(doc_word)
print(book_titles[75])
doc_topic[75]

The Merry Adventures of Robin Hood


array([0.        , 0.        , 0.        , 0.68361266, 0.        ,
       0.        , 0.00688305])

In [920]:
display_topics(nmf_model, tfidf.get_feature_names(), 10)


Topic  0
sister, father, dear, mother, lady, manner, brother, reply, pleasure, feeling

Topic  1
power, opinion, nature, subject, government, law, hath, common, sect, society

Topic  2
white, black, street, window, strange, case, glance, yes, earth, sir

Topic  3
thou, thee, thy, hath, unto, doth, hast, thyself, thou art, thine

Topic  4
ship, sea, captain, boat, island, shore, sail, voyage, kill, king

Topic  5
slave, slavery, master, dollar, tenement, negro, black, white, mother, labor

Topic  6
boy, oh, girl, dear, yes, mother, lady, laugh, sir, really


In [901]:
num_clusters = 10
km = KMeans(n_clusters=num_clusters,random_state=10,n_init=1) # n_init, number of times the K-mean algorithm will run
km.fit(doc_topic)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=10, n_init=1, n_jobs=None, precompute_distances='auto',
    random_state=10, tol=0.0001, verbose=0)

In [902]:
group_labels = km.labels_
for group,title in zip(group_labels, book_titles):
    if group == 6:
        print(title, group)

A Tale of Two Cities 6
The Yellow Wallpaper 6
Dracula 6
The Picture of Dorian Gray 6
Dubliners 6
Ulysses 6
Siddhartha 6
A Christmas Carol 6


## LDA

In [ ]:
# gensim
from gensim import corpora, models, similarities, matutils

# sklearn
from sklearn import datasets
from sklearn.feature_extraction.text import CountVectorizer

# logging for gensim (set to INFO)
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [ ]:
count_vectorizer = CountVectorizer(ngram_range=(1, 2),  
                                   stop_words='english')

doc_word_for_gen = count_vectorizer.fit_transform(lemmed_text).transpose()



In [ ]:
doc_word_for_gen.shape

In [ ]:
doc_word_for_gen.shape

In [ ]:
import pandas as pd

pd.DataFrame(doc_word_for_gen.toarray(), count_vectorizer.get_feature_names())[300:400]

In [ ]:
corpus = matutils.Sparse2Corpus(doc_word_for_gen)

In [ ]:
id2word = dict((v, k) for k, v in count_vectorizer.vocabulary_.items())

In [ ]:
len(id2word)

In [ ]:
lda = models.LdaModel(corpus=corpus, num_topics=7, id2word=id2word, passes=5)

In [ ]:
lda.print_topics()

# need to take out thee thy hath yes etc

# Need to make first category less important